In [50]:
from scipy import stats
import math as m
from sklearn.neighbors.kde import KernelDensity
from sympy import *
from scipy.stats.kde import gaussian_kde
from scipy.stats import norm
import numpy as np
import pandas as pd 
from datetime import datetime as dt

In [6]:
work_dir = os.getcwd()

In [122]:
test_data = pd.read_csv(os.path.join(work_dir, "data", "output_weekly.csv"))
newDate1 = []
for i in range(len(test_data)):
    newDate1.append(dt.strptime(test_data['Date'][i], "%Y/%m/%d").strftime('%Y/%m/%d'))
test_data["Date"] = newDate1

In [130]:
USG1M = pd.read_csv(os.path.join(work_dir, "Risk_free_rate", "DGS1MO.csv"))

newDate = []
for i in range(len(USG1M)):
    date_str =  str(USG1M['Time'][i])[:4] + '-' + str(USG1M['Time'][i])[4:6] + '-' + str(USG1M['Time'][i])[6:]
    newDate.append(dt.strptime(date_str, '%Y-%m-%d').strftime('%Y/%m/%d'))
USG1M["Date"] = newDate

weekly_return = []
for i in range(len(USG1M)):
    try:
         weekly_return.append(float(USG1M["DGS1MO"][i])/100/52)
    except:
        weekly_return.append(0)
USG1M["DGS1MO"] = weekly_return
USG1M.head()

,Time,DGS1MO,Date
0,20140509,0.000004,2014/05/09
1,20140512,0.000004,2014/05/12
2,20140513,0.000006,2014/05/13
3,20140514,0.000004,2014/05/14
4,20140515,0.000002,2014/05/15


In [144]:
weekly_ret = []
last_week_pos = 0
cur_week_pos = 0

for i in range(len(USG1M)):
    
    if(USG1M["Date"][i] in newDate1):
        if(last_week_pos == 0): #base case
            last_week_pos = i
            cur_week_pos = i
            weekly_ret.append(np.nan)
        else:
            last_week_pos = cur_week_pos
            cur_week_pos = i
            weekly_ret.append(sum(USG1M['DGS1MO'][last_week_pos:cur_week_pos]))
    else:
        weekly_ret.append(np.nan)

USG1M['Weekly_Rf'] = weekly_ret 
USG1M.head()
new_USG1M = USG1M.loc[:, "Date":"Weekly_Rf"]

In [146]:
Results = test_data.merge(new_USG1M, on = "Date")
Results.tail(5)
Results.to_csv(os.path.join(work_dir, "data", "DGS1MO.csv"))

,Date,FPA,FPXI,IQDE,IQDY,CHIQ,GEM,ADRA,PAF,IEMG,...,REMX,VEU,VXUS,AXJL,EMCG,XSOE,DNL,IHDG,DBAP,Weekly_Rf
156,2019/02/19,0.011000,0.007888,-0.000452,0.006606,0.031839,0.015394,0.005244,0.022616,0.015303,...,0.028552,0.007872,0.007794,0.012747,0.015221,0.017513,0.011044,0.009631,0.014552,0.002338
157,2019/02/25,-0.022293,0.004030,-0.002704,0.000000,0.000000,-0.021341,0.014020,-0.007989,-0.022958,...,-0.018405,-0.001197,-0.002695,-0.016924,-0.020693,-0.020451,-0.002174,0.005716,-0.001825,0.001867
158,2019/03/04,-0.021789,-0.023022,-0.014053,-0.026509,-0.043293,-0.019662,-0.019218,0.017403,-0.018757,...,-0.028858,-0.016413,-0.016812,-0.012159,-0.012826,-0.022199,-0.020283,-0.008199,-0.001462,0.002342
159,2019/03/11,0.011258,0.002019,0.016408,0.019056,-0.002489,0.014801,0.000000,-0.004677,0.016047,...,0.000000,0.016905,0.016511,0.010659,0.010023,0.013418,0.019593,0.011720,0.003266,0.002350
160,2019/03/18,-0.000978,0.010345,-0.007576,-0.004019,-0.022045,-0.006020,NaN,0.000000,-0.007242,...,-0.010638,0.003343,0.002660,-0.004425,0.002669,0.004481,-0.002143,-0.000312,0.000000,0.002356


In [41]:
Omega_week = pd.DataFrame()
temp_etf = test_data['SPX_Return'].dropna()
for i in range(156, len(temp_etf) + 1):
    x_value = np.linspace(-0.5,0.5,1000)
    etf_gqussian = gaussian_kde(temp_etf.iloc[i-156:i])
    L = USG1M.loc[temp_etf.iloc[i-156:i].index[-1]]

In [ ]:
Omega_week = pd.DataFrame()
columns_name = []
for k in range(etf_data_week_return.shape[1]):
    print(k)
    temp_etf=etf_data_week_return.iloc[:,k].dropna()
    omega=[]
    if len(temp_etf)>=156:
        columns_name.append(etf_data_week_return.columns[k])
    for i in range(156,len(temp_etf)+1):
        probDensityFun = gaussian_kde(temp_etf.iloc[i-156:i]-1)
        x=np.linspace(-0.5,0.5,10000)
        L=USG3M.loc[temp_etf.iloc[i-156:i].index[-1]]
        p=(probDensityFun(x)/probDensityFun(x).sum())
        p=p.cumsum()
        p=pd.DataFrame(p,index=x)
        omega.append(((1-p).loc[L/100/52:].sum())/(p.loc[:L/100/52].sum()))
    omega=pd.DataFrame(omega,index=temp_etf.index[155:])
    omega_combine_week=pd.concat([omega_combine_week,omega],axis=1)
omega_combine_week.columns=columns_name